In [1]:
from helper import *

In [2]:
SUBSET = '20k'
DATA_DIR = 'data/'

In [3]:
DATA_FILENAME = '3ch_subset'
(X_train, y_train), (X_test, y_test) = \
    load_npz(DATA_DIR + DATA_FILENAME + SUBSET + '.npz')

((200000, 96, 96, 3), dtype('uint8')) ((200000,), dtype('uint8'))
((20000, 96, 96, 3), dtype('uint8')) ((20000,), dtype('uint8'))


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.losses import BinaryCrossentropy
from keras.layers import *
from keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.optimizers import SGD, Adam
import os

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
inputs = Input(shape=X_train[0].shape)
model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)
model.trainable = True
x = GlobalAveragePooling2D()(model.output)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 96, 96, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 96, 96, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 97, 97, 3)    0           normalization[0][0]              
______________________________________________________________________________________________

In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=15,
                    verbose=2, 
                    validation_data=(X_test, y_test)
                   )

Epoch 1/15


In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=15,
                    verbose=2, 
                    validation_data=(X_test, y_test)
                   )

In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=SGD(learning_rate=0.005),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=15,
                    verbose=2, 
                    validation_data=(X_test, y_test)
                   )